In [1]:
import os
import re
import sys
import json

import pandas
import jieba
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col


In [2]:
jieba.set_dictionary("/home/workspace/data/dict_tw.txt")

In [3]:
spark = (SparkSession
         .builder
         .master("spark://spark:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate()
         )

sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/18 09:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
f = open("/home/workspace/data/input/input.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [6]:
newJson = str(data)
df = spark.read.json(sc.parallelize([newJson]))


In [7]:
# df.show(truncate = False)

In [8]:
df.printSchema()

root
 |-- content: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- title: string (nullable = true)



In [12]:

def regular(w):
    line = re.findall('[\u4e00-\u9fa5]+', w)
    if len(line) > 0:
        return line


def get_cut(content):
    seg_list = jieba.cut(content, cut_all=True, HMM=True)
    return seg_list


def text_cleaning(paragraph):
    merge_words = []
    words = get_cut(paragraph)
    for w in words:
        if len(w) > 1:
            w = regular(w)
            if w is not None:
                merge_words.extend(w)
    return "\n".join(merge_words)

In [13]:

titleDf = df.select(
    col("link_id"),
    col("title")
)
wordDf = df.select(
    col("content"),
    col("link_id")
).rdd.map(
    lambda x: (x[0], x[1])
).flatMap(
    lambda x: ([(w, x[1]) for w in text_cleaning(x[0]).split("\n")])
).map(
    lambda x: (x, 1)
).reduceByKey(
    lambda x, y: x + y
).map(
    lambda x: (x[0][0], x[0][1], x[1])
).toDF(["word", "index", "count"])






In [1]:
# titleDf.show(truncate = False)
# wordDf.show(truncate = False)